## 18. Random access with `seek()` / `tell()`

Files are like byte arrays. `f.seek(offset, whence)` moves the cursor; `whence` 0 = start, 1 = current, 2 = end. Useful for fixed‑size records or editing headers.

```python
with open('demo.bin','wb+') as f:
    f.write(b'HELLOworld')
    f.seek(5)
    f.write(b'WORLD')
    f.seek(0)
    print(f.read())  # b'HELLOWORLD'
```

### Quick check

1. `tell()` returns:
  a. bytes remaining   b. current offset

2. True / False `seek(-1, 2)` moves to last byte.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 19. Memory‑mapped files (`mmap`)

`mmap.mmap()` maps a file into memory → random access to big files without reading all bytes. Changes in slice mutate file. Works only on real files, not stdin.

```python
import mmap, os
with open('big.txt','wb+') as f:
    f.truncate(10)
    mm = mmap.mmap(f.fileno(), 0)
    mm[0:4] = b'data'
    mm.flush()
    mm.close()
print(open('big.txt','rb').read())
```

### Quick check

1. Memory maps can exceed RAM?
  a. yes   b. no

2. True / False Closing mmap writes changes to disk.

<details><summary>Answer key</summary>

1. **a** – OS pages in/out.
2. **True**.

</details>

## 20. File locking basics

Multiple writers risk corruption. Use OS locks: `fcntl.flock` on Unix, `msvcrt.locking` on Windows, or cross‑platform libs like `portalocker`.

```python
import portalocker, time
with open('lock.txt','w') as f:
    portalocker.lock(f, portalocker.LOCK_EX)
    f.write('exclusive')
    time.sleep(1)
    portalocker.unlock(f)
```

### Quick check

1. Advisory lock means:
  a. kernel enforced   b. cooperative

2. True / False Windows and Unix use same lock API.

<details><summary>Answer key</summary>

1. **b** – other processes must honor.
2. **False**.

</details>

## 21. Streaming large files with chunked reads

Read in chunks to avoid memory blowup: `for chunk in iter(lambda: f.read(8192), b''):`. Combine with gzip or upload APIs.

```python
total=0
with open('big.txt','rb') as f:
    for chunk in iter(lambda: f.read(4096), b''):
        total += len(chunk)
print('bytes', total)
```

### Quick check

1. Chunk read loop stops when `read()` returns:
  a. None   b. b''

2. True / False `readinto()` fills existing buffer to avoid allocation.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 22. Compression on the fly

`gzip.open`, `bz2.open`, `lzma.open` act like `open()` but compress  transparently. Or use `tarfile`/`zipfile` for archives.

```python
import gzip
with gzip.open('hello.gz','wt') as g:
    g.write('hello world')
print(gzip.open('hello.gz','rt').read())
```

### Quick check

1. Writing text to gzip requires mode:
  a. 'wb'   b. 'wt'

2. True / False `zipfile` preserves file permissions by default.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – permissions lost unless set.

</details>

## 23. Symbolic vs. hard links

Symlink points to pathname; hard link points to inode (same file entry). Deleting original breaks symlink but not hard link.

```bash
ln file1 original.txt
ln -s original.txt alias.txt
```

### Quick check

1. Which can cross filesystem boundaries?
  a. hard link   b. symlink

2. True / False Removing original file keeps hard link data.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – inode lives until last link removed.

</details>